In [31]:
# Test if gpu-support is working.
import xgboost as xgb
from sklearn.datasets import load_boston

boston = load_boston()

# XGBoost API example
params = {'tree_method': 'gpu_hist', 'max_depth': 3, 'learning_rate': 0.1}
dtrain = xgb.DMatrix(boston.data, boston.target)
xgb.train(params, dtrain, evals=[(dtrain, "train")])

# sklearn API example
gbm = xgb.XGBRegressor(silent=False, n_estimators=10, tree_method='gpu_hist')
gbm.fit(boston.data, boston.target, eval_set=[(boston.data, boston.target)])

[0]	train-rmse:21.60208
[1]	train-rmse:19.55555
[2]	train-rmse:17.71453
[3]	train-rmse:16.06071
[4]	train-rmse:14.57054
[5]	train-rmse:13.23501
[6]	train-rmse:12.03721
[7]	train-rmse:10.94916
[8]	train-rmse:9.98378
[9]	train-rmse:9.10604
[17:54:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:17.06433
[1]	validation_0-rmse:12.27140
[2]	validation_0-rmse:8.92523
[3]	validation_0-rmse:6.56554
[4]	validation_0-rmse:4.88678
[5]	validation_0-rmse:3.70939
[6]	validation_0-rmse:2.89225
[7]	validation_0-rmse:2.33943
[8]	validation_0-rmse:1.94412
[9]	validation_0-rmse:1.66867


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=False,
             subsample=1, tree_method='gpu_hist', validate_parameters=1,
             verbosity=None)

In [45]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv('session_buy_data_2_14.csv')
test = pd.read_csv('session_buy_data_1.csv')
buy=data.groupby('buy').get_group(1)
nobuy=data.groupby('buy').get_group(0)
train = data
x_train = train[['item_id','clicks']]
y_train = train['buy']
x_test = test[['item_id','clicks']]
y_test = test['buy']
#x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, random_state = 33)
xgbc = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=24.5, seed=None,
       silent=True, subsample=1,tree_method='gpu_hist')
xgbc.fit(x_train, y_train)
print("Accu:",xgbc.score(x_test,y_test))
a = xgbc.predict_proba(x_test)
test['pred'] = xgbc.predict_proba(x_test)[:,1]
print(test)

[19:16:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:16:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accu: 0.6715727269002648
         session    item_id  clicks  buy      pred
0              1  214536500       1    0  0.273346
1              1  214536506       1    0  0.273346
2              1  214577561       1    0  0.253401
3              1  214536502       1    0  0.273346
4            128  21482661

In [53]:
test.groupby('buy').get_group(1)#.sort_values(by=['pred'])

,session,sales,duration,clicks,items,buy,prob
4,669,0,50.954,3,1,1,0.554296
11,1821,0,48.767,2,1,1,0.551624
13,2077,0,140.128,5,3,1,0.588169
45,7197,0,372.736,3,1,1,0.701910
55,8861,0,2333.206,6,3,1,0.789269
...,...,...,...,...,...,...,...
72209,11553437,0,168.321,5,3,1,0.643471
72217,11554717,0,197.962,2,1,1,0.576822
72228,11556509,3,453.607,3,3,1,0.455293
72237,11557917,2,358.004,2,1,1,0.661919


In [49]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.DataFrame()
for i in range(0,29):
    rpath = "train_data/train_data_p"+str(i)+".csv"
    #print("[TRAIN] Reading:",rpath)
    d0 = pd.read_csv(rpath)
    data = pd.concat([data,d0])

test = pd.read_csv('train_data/train_data_p29.csv')

x_train = data[['sales','duration','items','clicks']]
y_train = data['buy']
x_test = test[['sales','duration','items','clicks']]
y_test = test['buy']

print("Trainning data size:",x_train.shape)
print("Testing data size:",x_test.shape)

buy = data.groupby('buy')
ybuy = buy.get_group(1)
nbuy = buy.get_group(0)
print("P data size:",ybuy.shape)
print("N data size:",nbuy.shape)

Trainning data size: (2095656, 4)
Testing data size: (72264, 4)
P data size: (115125, 6)
N data size: (1980531, 6)


In [37]:
print("P data size:",ybuy.shape[0])
print("N data size:",nbuy.shape[0])
print("ratio:",nbuy.shape[0]/ybuy.shape[0])

P data size: 115125
N data size: 1980531
ratio: 17.20330944625407


In [52]:
#train_new = pd.concat([ybuy,nbuy])
train_new = data

x_train = train_new[['sales','duration','items','clicks']]
y_train = train_new['buy']
x_test = test[['sales','duration','items','clicks']]
y_test = test['buy']

print("Trainning data size:",x_train.shape)
print("Testing data size:",x_test.shape)

#xgbc = xgb.XGBClassifier(
 #learning_rate =0.1,n_estimators=1000,max_depth=8,
 #min_child_weight=1,gamma=0,subsample=0.8,
 #colsample_bytree=0.8,objective= 'binary:logistic',
 #scale_pos_weight=17.2,seed=0,tree_method='gpu_hist')
xgbc = xgb.XGBClassifier(base_score=0.5, booster='gbtree',
                         gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=0.1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=0, scale_pos_weight=17.2, seed=None,
       silent=True, subsample=0.5,tree_method='gpu_hist')
xgbc.fit(x_train, y_train)
print("Accuracy:",xgbc.score(x_test,y_test))
test['prob']=xgbc.predict_proba(x_test)[:,1]
print("Actually buys:",test.groupby('buy').get_group(1).head(20))

Trainning data size: (2095656, 4)
Testing data size: (72264, 4)


c:\python39\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:20:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:20:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.7001411491198937
Actually buys:      session  sales  duration  clicks  items  buy      prob
4        669      0    50.954       3      1    1  0.554296
11      1821      0    48.767       2      1    1  0.551624
13      2077      0   140.128       5      3    1  0.588169
45      7197      0   

In [20]:
print("Actually buys:\n",test.groupby('buy').get_group(1).sort_values(by=['prob']).head(50))
print("Actually buys:\n",test.groupby('buy').get_group(1).sort_values(by=['prob'],ascending=False).head(50))

Actually buys:
         session  sales  duration  clicks  items  buy      prob
51596   8255389      5    51.808       5      2    1  0.020011
72079  11532701      0     3.412       3      3    1  0.023472
72087  11533981     10   168.055      10     10    1  0.056748
12981   2076957      0    10.241       3      2    1  0.057155
46745   7479197      4   671.583      27     25    1  0.061116
33067   5290781      2     4.005       2      2    1  0.072340
19408   3105309      0    63.663       4      4    1  0.073025
10345   1655197      0   842.263      32     17    1  0.075592
49361   7897757      3     8.913       3      3    1  0.087291
72038  11526173      1     0.308       3      3    1  0.088625
40587   6493981      6  5846.244       6      2    1  0.088736
22640   3622429      0     0.062       2      2    1  0.102651
40079   6412701      0   103.786       4      4    1  0.105455
31028   4964509      0     0.769       2      1    1  0.108145
53478   8556573      0     0.143       

In [126]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional     scklearn functions
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV   #Perforing grid search

def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])

    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['buy'],eval_metric='auc')
    
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['buy'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['buy'], dtrain_predprob))
                
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')
    
target = 'buy'
IDcol = 'session'

predictors = [x for x in train.columns if x not in [target,IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27,
 tree_method='gpu_hist')

modelfit(xgb1, train, predictors)


Model Report
Accuracy : 0.799
AUC Score (Train): 0.882039


TypeError: 'NoneType' object is not callable